# Instructions

In this tutorial, we will perform multi-label classification using an ECG-FM model finetuned on the [MIMIC-IV-ECG v1.0 dataset](https://physionet.org/content/mimic-iv-ecg/1.0/). It outlines the data and model loading, as well as inference, same-sample prediction aggregation, and visualizations for embeddings and saliency maps.

ECG-FM was developed in collaboration with the [fairseq_signals](https://github.com/Jwoo5/fairseq-signals) framework, which implements a collection of deep learning methods for ECG analysis.

This is segment the ECG into inputs of 5 s and perform a label-specific aggregation of the predictions from each sample

This document serves largely as a quickstart introduction. Much of this functionality is also available via the [fairseq-signals scripts](https://github.com/bowang-lab/ECG-FM/blob/main/notebooks/infer_cli.ipynb), as well the [ECG-FM scripts](https://github.com/bowang-lab/ECG-FM/tree/main/scripts).

## Installation

Begin by cloning [fairseq_signals](https://github.com/Jwoo5/fairseq-signals) and refer to the installation section in the top-level README. For example, the following commands are sufficient at the present moment:
```
# Creating `fairseq` environment:
conda create --name fairseq python=3.10.6
source activate fairseq
git clone https://github.com/Jwoo5/fairseq-signals
cd fairseq-signals
python3 -m pip install pip==24.0
python3 -m pip install -e .
```

In [ ]:
# You may require the following imports depending on what functionality you run
!pip install huggingface-hub
!pip install pandas
!pip install ecg-transform==0.1.3
!pip install umap-learn
!pip install plotly

  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.3 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
numba 0.60.0 requires numpy<2.1,>=1.22, but you have numpy 2.3.3 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.3 which is incompatible.
opencv-contrib-python 4.12.0.88 requir

  Using cached numpy-2.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
Using cached numpy-2.0.2-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.2 MB)
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 447, in run
    conflicts = self._determine_conflicts(to_install)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pip/_internal/commands/install.py", line 578, in _determine_conflicts
    return check_install_conflicts(to_install)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [10]:
import os

root = os.path.dirname(os.getcwd())

In [12]:
# 1) Get fairseq-signals (dependency of ECG-FM)
!git clone https://github.com/Jwoo5/fairseq-signals.git
%cd fairseq-signals

# Install fairseq-signals in editable mode + common deps
!pip install --upgrade pip
!pip install -e .
!pip install omegaconf==2.* hydra-core==1.* pandas scipy wfdb pyarrow einops

# 2) Get ECG-FM
%cd /content
!git clone https://github.com/bowang-lab/ECG-FM.git
%cd ECG-FM

# 3) (Optional) pull model weights if the notebook expects them locally
# You can also grab weights from HF and place under ./ckpts/
# !wget -O ckpts/physionet_finetuned.pt <YOUR-LINK>



Cloning into 'fairseq-signals'...
remote: Enumerating objects: 4426, done.
remote: Counting objects: 100% (384/384), done.
remote: Compressing objects: 100% (160/160), done.
remote: Total 4426 (delta 248), reused 269 (delta 218), pack-reused 4042 (from 1)
Receiving objects: 100% (4426/4426), 5.38 MiB | 8.76 MiB/s, done.
Resolving deltas: 100% (2901/2901), done.
/content/ECG-FM/fairseq-signals
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Obtaining file:///content/ECG-FM/fairseq-signals
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 83.4 MB/s  0:00:00
  Buildi

## Download checkpoints

Checkpoints are available on [HuggingFace](https://huggingface.co/wanglab/ecg-fm). The finetuned model be downloaded using the following command:

In [13]:
import os
from huggingface_hub import hf_hub_download

_ = hf_hub_download(
    repo_id='wanglab/ecg-fm',
    filename='mimic_iv_ecg_finetuned.pt',
    local_dir=os.path.join(root, 'ckpts'),
)

# Inference

In [15]:
ckpt_path: str = os.path.join(root, 'ckpts/mimic_iv_ecg_finetuned.pt')
assert os.path.isfile(ckpt_path)

device: str = 'cuda'
batch_size: int = 16
num_workers: int = 0

extract_saliency: bool = True

In [17]:
from typing import Any, List

def to_list(obj: Any) -> List[Any]:
    if isinstance(obj, list):
        return obj

    if isinstance(obj, (np.ndarray, set, dict)):
        return list(obj)

    return [obj]

file_paths = [
    os.path.join(root, '/content/ECG-FM/data', file) for file in \
    os.listdir(os.path.join(root, '/content/ECG-FM/data'))
]
file_paths = to_list(file_paths)
file_paths

['/content/ECG-FM/data/mimic_iv_ecg',
 '/content/ECG-FM/data/manifests',
 '/content/ECG-FM/data/code_15']

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Prepare data

To simplify this tutorial, we have processed a sample of 10 ECGs (14 5s segments) from the [CODE-15% v1.0.0 dataset](https://zenodo.org/records/4916206/) using our [end-to-end data preprocessing pipeline](https://github.com/Jwoo5/fairseq-signals/tree/master/scripts/preprocess/ecg). Its README is also helpful if looking to perform inference using your own dataset, where there are already preprocessing scripts implemented for several public datasets.

In [18]:
from typing import List
from itertools import chain

from scipy.io import loadmat

import numpy as np

import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader

from ecg_transform.inp import ECGInput, ECGInputSchema
from ecg_transform.sample import ECGMetadata, ECGSample
from ecg_transform.t.base import ECGTransform
from ecg_transform.t.common import (
    HandleConstantLeads,
    LinearResample,
    ReorderLeads,
)
from ecg_transform.t.scale import Standardize
from ecg_transform.t.cut import SegmentNonoverlapping

class ECGFMDataset(Dataset):
    def __init__(
        self,
        schema,
        transforms,
        file_paths,
    ):
        self.schema = schema
        self.transforms = transforms
        self.file_paths = file_paths

    def __len__(self):
        return len(self.file_paths)

    def __getitem__(self, idx):
        mat = loadmat(self.file_paths[idx])
        metadata = ECGMetadata(
            sample_rate=int(mat['org_sample_rate'][0, 0]),
            num_samples=mat['feats'].shape[1],
            lead_names=['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6'],
            unit=None,
            input_start=0,
            input_end=mat['feats'].shape[1],
        )
        metadata.file = self.file_paths[idx]
        inp = ECGInput(mat['feats'], metadata)
        sample = ECGSample(
            inp,
            self.schema,
            self.transforms,
        )
        source = torch.from_numpy(sample.out).float()

        return source, inp

def collate_fn(inps):
    sample_ids = list(
        chain.from_iterable([[inp[1]]*inp[0].shape[0] for inp in inps])
    )
    return torch.concatenate([inp[0] for inp in inps]), sample_ids

def file_paths_to_loader(
    file_paths: List[str],
    schema: ECGInputSchema,
    transforms: List[ECGTransform],
    batch_size = 64,
    num_workers = 7,
):
    dataset = ECGFMDataset(
        schema,
        transforms,
        file_paths,
    )

    return DataLoader(
        dataset,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=True,
        sampler=None,
        shuffle=False,
        collate_fn=collate_fn,
        drop_last=False,
    )

In [21]:
import pandas as pd

ECG_FM_LEAD_ORDER = ['I', 'II', 'III', 'aVR', 'aVL', 'aVF', 'V1', 'V2', 'V3', 'V4', 'V5', 'V6']
SAMPLE_RATE = 500
N_SAMPLES = SAMPLE_RATE*5

label_def = pd.read_csv(
    os.path.join(root, '/content/ECG-FM/data/mimic_iv_ecg/labels/label_def.csv'),
     index_col='name',
)
label_names = label_def.index.to_list()
label_names

['Poor data quality',
 'Sinus rhythm',
 'Premature ventricular contraction',
 'Tachycardia',
 'Ventricular tachycardia',
 'Supraventricular tachycardia with aberrancy',
 'Atrial fibrillation',
 'Atrial flutter',
 'Bradycardia',
 'Accessory pathway conduction',
 'Atrioventricular block',
 '1st degree atrioventricular block',
 'Bifascicular block',
 'Right bundle branch block',
 'Left bundle branch block',
 'Infarction',
 'Electronic pacemaker']

In [22]:
AGG_METHODS = {
    'Poor data quality': 'max',
    'Sinus rhythm': 'mean',
    'Premature ventricular contraction': 'max',
    'Tachycardia': 'mean',
    'Ventricular tachycardia': 'max',
    'Supraventricular tachycardia with aberrancy': 'max',
    'Bradycardia': 'mean',
    'Infarction': 'mean',
    'Atrioventricular block': 'mean',
    'Right bundle branch block': 'mean',
    'Left bundle branch block': 'mean',
    'Electronic pacemaker': 'max',
    'Atrial fibrillation': 'mean',
    'Atrial flutter': 'mean',
    'Accessory pathway conduction': 'mean',
    '1st degree atrioventricular block': 'mean',
    'Bifascicular block': 'mean',
}

ECG_FM_SCHEMA = ECGInputSchema(
    sample_rate=SAMPLE_RATE,
    expected_lead_order=ECG_FM_LEAD_ORDER,
    required_num_samples=N_SAMPLES,
)

ECG_FM_TRANSFORMS = [
    ReorderLeads(
        expected_order=ECG_FM_LEAD_ORDER,
        missing_lead_strategy='raise',
    ),
    LinearResample(desired_sample_rate=SAMPLE_RATE),
    HandleConstantLeads(strategy='zero'),
    Standardize(),
    SegmentNonoverlapping(segment_length=N_SAMPLES),
]

loader = file_paths_to_loader(
    file_paths,
    ECG_FM_SCHEMA,
    ECG_FM_TRANSFORMS,
    batch_size=batch_size,
    num_workers=num_workers,
)

## Load model

In [24]:
# Uninstall numpy
!pip uninstall -y numpy

Found existing installation: numpy 2.3.3
Uninstalling numpy-2.3.3:
  Successfully uninstalled numpy-2.3.3


In [33]:
from typing import Dict, List, Optional, Tuple, Type, Union
from collections import OrderedDict

#import numpy as np
import pandas as pd

import torch

from fairseq_signals.models import build_model_from_checkpoint
from fairseq_signals.models.classification.ecg_transformer_classifier import (
    ECGTransformerClassificationModel
)

ModuleNotFoundError: No module named 'fairseq_signals'

In [34]:
# 1) Get fairseq-signals (dependency of ECG-FM)
!git clone https://github.com/Jwoo5/fairseq-signals.git
%cd fairseq-signals

# Install fairseq-signals + common deps
!pip install --upgrade pip
!pip install .
!pip install omegaconf==2.* hydra-core==1.* pandas scipy wfdb pyarrow einops

# 2) Get ECG-FM
%cd /content
!git clone https://github.com/bowang-lab/ECG-FM.git
%cd ECG-FM

# 3) (Optional) pull model weights if the notebook expects them locally
# You can also grab weights from HF and place under ./ckpts/
# !wget -O ckpts/physionet_finetuned.pt <YOUR-LINK>

fatal: destination path 'fairseq-signals' already exists and is not an empty directory.
/content/ECG-FM/fairseq-signals
Processing /content/ECG-FM/fairseq-signals
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (62 kB)
Using cached numpy-2.3.3-cp312-cp312-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl (16.6 MB)
  Created wheel for fairseq_signals: filename=fairseq_signals-1.0.0a0+571a124-cp312-cp312-linux_x86_64.whl size=894473 sha256=acd24e8d429a593ea4fc9bac1465291733d9c45dece805087f20b929ae9edf61
  Stored in directory: /root/.cache/pip/wheels/bb/f3/5b/d4ccc487af1822d33929632234306a1022e0840f7ad388214c
Successfully built fairseq_signals
  Attempting uninstall: fairseq_signals
    Found existing installation: fairseq_signals 1.0.0a0+571a124
    Uninstalling fairs

/content
fatal: destination path 'ECG-FM' already exists and is not an empty directory.
/content/ECG-FM


In [ ]:
model: ECGTransformerClassificationModel = build_model_from_checkpoint(
    checkpoint_path=ckpt_path
)

# Forcibly enable the return of attention weights for saliency maps
if extract_saliency:
    model.encoder.encoder.need_weights = extract_saliency
    for layer in model.encoder.encoder.layers:
        layer.need_weights = extract_saliency

model.eval()
model.to(device)

## Infer

In [ ]:
def encoder_out_to_emb(x, device='cpu'):
    # fairseq_signals/models/classification/ecg_transformer_classifier.py
    return torch.div(x.sum(dim=1), (x != 0).sum(dim=1))

def infer(
    model,
    loader,
    device,
    extract_saliency: bool = True,
):
    inps = []
    sources = []
    logits = []
    embs = []
    saliency = []
    file_names = []
    for source, inp in loader:
        source = source.to(device)
        out = model(source=source)
        inps.extend(inp)
        sources.append(source)
        logits.append(out['out'])
        embs.append(encoder_out_to_emb(out['encoder_out']))
        saliency.append(out['saliency'])
        file_names.extend([i.meta.file for i in inp])

    # Handle predictions
    pred = torch.sigmoid(torch.concatenate(logits)).detach().cpu().numpy()
    pred = pd.DataFrame(pred, columns=label_names, index=file_names)

    results = {
        'inps': inps,
        'sources': torch.concatenate(sources).detach().cpu().numpy(),
        'embs': torch.concatenate(embs).detach().cpu().numpy(),
        'pred': pred,
    }

    # Handle saliency
    if extract_saliency:
        saliency = torch.concatenate(saliency).detach()
        attn = saliency[:, -1] # Consider only the last attention layer
        results['attn_max'] = attn.max(axis=2).values.squeeze().cpu().detach().numpy()

    return results

In [ ]:
results = infer(model, loader, device)

In [ ]:
pred = results['pred']
print(f"Number of 5 s segment predictions: {len(pred)}.")
pred

# Result handling

## Prediction aggregation

In [ ]:
pred_agg = pred.groupby(pred.index).agg(AGG_METHODS).astype(float)
print(f"Number of sample-aggregated predictions: {len(pred_agg)}.")
pred_agg

## Visualizing embeddings

In [ ]:
import matplotlib.pyplot as plt
import umap

reducer = umap.UMAP(n_neighbors=3, min_dist=0.1, n_components=2, random_state=42)
embs_2d = reducer.fit_transform(results['embs'])

# Generate a color map
sample_identifier = pred.index.to_series()
unique_values = sample_identifier.unique()
colors = plt.colormaps.get_cmap('tab20')  # Use a colormap with enough distinct colors
color_map = {val: colors(i) for i, val in enumerate(unique_values)}
colored_items = sample_identifier.map(color_map)

# Plot the 2D UMAP visualization
plt.scatter(
    embs_2d[:, 0],
    embs_2d[:, 1],
    s=30,
    alpha=0.9,
    color=colored_items.values,
    rasterized=True,
)

# Remove axis labels and grid
plt.xticks([])
plt.yticks([])
plt.grid(False)

More fitting when visualizing many embeddings:
```
import matplotlib.pyplot as plt
import umap
reducer = umap.UMAP(n_neighbors=15, min_dist=0.1, n_components=2, random_state=42) # Better when there are more embeddings

# Plot the 2D UMAP visualization
plt.scatter(
    embs_2d[:, 0],
    embs_2d[:, 1],
    s=1,
    alpha=0.9,
    rasterized=True,
)

# Remove axis labels and grid
plt.xticks([])
plt.yticks([])
plt.grid(False)
```

## Saliency maps

In [ ]:
from typing import Tuple

import numpy as np

from scipy.ndimage import map_coordinates

import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
sample_idx = 0

saliency_lead = 'II'
lead_ind = ECG_FM_LEAD_ORDER.index(saliency_lead)

In [ ]:
signal = results['sources'][sample_idx, lead_ind]
attn_max = results['attn_max'][sample_idx]

In [ ]:
def blend_colors_hex(start_color: str, end_color: str, activations: np.ndarray) -> np.ndarray:
    """
    Blends between two colors based on an array of blend factors.

    Parameters
    ----------
    start_color : str
        Hexadecimal color code for the start color.
    end_color : str
        Hexadecimal color code for the end color.
    activations : np.ndarray
        An array of blend factors where 0 corresponds to the start color and 1 to the end color.

    Returns
    -------
    np.ndarray
        An array of hexadecimal color codes resulting from the blends.

    Raises
    ------
    ValueError
        If any of the input blend factors are not within the range [0, 1].
    """
    if np.any((activations < 0) | (activations > 1)):
        raise ValueError("All blend factors must be between 0 and 1.")

    # Convert hexadecimal to RGB
    def hex_to_rgb(hex_color: str) -> Tuple[int]:
        return tuple(int(hex_color[i: i+2], 16) for i in (1, 3, 5))

    # Get RGB tuples
    start_rgb = np.array(hex_to_rgb(start_color))
    end_rgb = np.array(hex_to_rgb(end_color))

    # Blend RGB values
    blended_rgb = np.outer(1 - activations, start_rgb) + np.outer(activations, end_rgb)

    # Convert blended RGB back to hex codes
    return blended_rgb / 255

def colored_line_segments(data: np.ndarray, colors: np.ndarray, ax=None, **kwargs):
    """
    Plots line segments based on the provided data points, with each segment
    colored according to the corresponding color specification in `colors`.

    Parameters
    ----------
    data : np.ndarray
        Array of y-values for the line segments.
    colors : np.ndarray
        Array of colors, each color applied to the corresponding line segment
        between points i and i+1.

    Raises
    ------
    ValueError
        If the `colors` array does not have exactly one less element than the `data` array,
        as each segment needs a unique color.

    Returns
    -------
    None
    """
    if len(colors) != len(data) - 1:
        raise ValueError("Colors array must have one fewer elements than data array.")

    if ax is None:
        for i in range(len(data) - 1):
            plt.plot([i, i + 1], [data[i], data[i + 1]], color=colors[i], **kwargs)
    else:
        for i in range(len(data) - 1):
            ax.plot([i, i + 1], [data[i], data[i + 1]], color=colors[i], **kwargs)

def prep_saliency_values(attn_max, target_sample_length):
    # Resample to original sample size
    new_dims = [
        np.linspace(0, original_length-1, new_length) \
        for original_length, new_length in \
        zip(attn_max.shape, (target_sample_length - 1,))
    ]
    coords = np.meshgrid(*new_dims, indexing='ij')
    attn_max = map_coordinates(attn_max, coords)

    # Min-max normalization
    attn_max = attn_max - attn_max.min()
    attn_max = attn_max/attn_max.max()

    return attn_max

saliency_prepped = prep_saliency_values(
    attn_max.ravel(),
    attn_max.shape[0] * signal.shape[-1],
)
saliency_colors = blend_colors_hex('#0047AB', '#DC143C', saliency_prepped)
saliency_colors = (saliency_colors*255).astype(int)

# Define a custom colorscale from blue to red
colorscale = [[0, 'blue'], [1, 'red']]  # Simple gradient from blue to red

time = np.arange(2500)

# Create the figure
fig = go.Figure()
y_values = signal[:-1]
for i in range(len(y_values) - 1):
    fig.add_trace(
        go.Scatter(
            x=[time[i], time[i + 1]],
            y=[y_values[i], y_values[i + 1]],
            mode='lines',
            line=dict(color='rgb({},{},{})'.format(*saliency_colors[i]), width=2),
            showlegend=False  # Avoid cluttering the legend
        )
    )
fig['layout']['yaxis'].update(autorange = True)
fig['layout']['xaxis'].update(autorange = True)

fig.show()